In [6]:
import pandas as pd
import numpy as np
import itertools
import wikipedia
import re
import nltk

# sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

%matplotlib inline

## Wiki-Pedia API 

In [2]:
wikipedia.search(query='AAPL')

['AAPL',
 'Apple Inc.',
 'History of Apple Inc.',
 'American Association of Professional Landmen',
 'Option symbol',
 'Alpha Indexes',
 'Energy management (degree)',
 'I Am Rich',
 'Jeff Williams (Apple)',
 'Landman (oil worker)']

In [3]:
wikipage = wikipedia.page('Apple Inc.')
print(wikipage.content[:1000])

Apple Inc. is an American multinational technology company headquartered in Cupertino, California, that designs, develops, and sells consumer electronics, computer software, and online services. The company's hardware products include the iPhone smartphone, the iPad tablet computer, the Mac personal computer, the iPod portable media player, the Apple Watch smartwatch, the Apple TV digital media player, and the HomePod smart speaker. Apple's software includes the macOS and iOS operating systems, the iTunes media player, the Safari web browser, and the iLife and iWork creativity and productivity suites, as well as professional applications like Final Cut Pro, Logic Pro, and Xcode. Its online services include the iTunes Store, the iOS App Store and Mac App Store, Apple Music, and iCloud.
Apple was founded by Steve Jobs, Steve Wozniak, and Ronald Wayne in April 1976 to develop and sell Wozniak's Apple I personal computer. It was incorporated as Apple Computer, Inc. in January 1977, and sal

## LDA -- Topic Modeling

## sklearn implementation

## note:
1. In sklearn implementation: analyzer = tokenizer (implemented by re.compile(self.token_pattern).findall(doc)) + n_gram_processor (include stop words) (implemented in self._word_ngrams)
2. (CountVectorizer, TfidfVectorizer) Vectorize process in sklearn: create vocab (tokenize & analyze) -> sort words by count -> limits words (remove rare words and stop words) -> BOW representation
3. How tokenize n-grams in sklearn: first, define a method "tokenizer". Second pass "tokenizer" in CountVectorizer(stop_words='english', ngram_range=(n, n), tokenizer=tokenizer). Last, CountVectorizer(stop_words='english', ngram_range=(n, n), tokenizer=tokenizer).build_analyzer() is the n-gram tokenizer
4. LDA algo works better when it transforms from bag-of-words counts into a topic space of lower dimensionality; LDA is worse when it transform from Tfidf representation

In [7]:
## prepare sentences for this doc
sentences = nltk.sent_tokenize(wikipage.content)
## define tokenizer for sklearn model
# tokenizer = CountVectorizer(stop_words='english', ngram_range=(1, 1)).build_tokenizer()
# tokenizer = CountVectorizer(stop_words='english', ngram_range=(1, 1)).build_analyzer()
def tokenizer(string):
    # include stemming (full-package service)
    return preprocess_string(string)
## vectorize tokens into bag of word matrix
# vectorizer = CountVectorizer(stop_words='english', analyzer=tokenizer)
# vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 1), tokenizer=tokenizer)
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 1), tokenizer=tokenizer)
bow_mat = vectorizer.fit_transform(sentences)
## LDA transform
lda = LatentDirichletAllocation(n_components=5, n_jobs=40)
lda_mat = lda.fit_transform(bow_mat)
lda_mat

/Users/Kevin/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


array([[0.01215291, 0.63113412, 0.01184059, 0.01190728, 0.3329651 ],
       [0.00803294, 0.00809477, 0.00803257, 0.00804427, 0.96779545],
       [0.00744249, 0.00742993, 0.00743633, 0.00743891, 0.97025235],
       ...,
       [0.01540143, 0.01557173, 0.01559895, 0.01545711, 0.93797078],
       [0.02501256, 0.89964828, 0.02527366, 0.02500515, 0.02506035],
       [0.00590642, 0.00591167, 0.00589606, 0.9763359 , 0.00594995]])